# Routing by semantic similarity

With LCEL you can easily add custom routing logic to your chain to dynamically determine the chain logic based on user input. All you need to do is define a function that given an input returns a Runnable.

One especially useful technique is to use embeddings to route a query to the most relevant prompt. Here’s a very simple example.

In [1]:
import sys
import os
module_path = os.path.abspath(os.path.join('..'))
model_config_path = os.path.abspath(os.path.join('../custom_llms/'))
sys.path.insert(0, module_path)
sys.path.insert(0, model_config_path)

from custom_llms import (
    ZhipuAIEmbeddings,
    Zhipuai_LLM,
    load_api
)
api_key = load_api()
model = Zhipuai_LLM(zhipuai_api_key=api_key)

In [2]:
from langchain.prompts import PromptTemplate
from langchain.utils.math import cosine_similarity
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise and easy to understand manner. \
When you don't know the answer to a question you admit that you don't know.

Here is a question:
{query}"""

math_template = """You are a very good mathematician. You are great at answering math questions. \
You are so good because you are able to break down hard problems into their component parts, \
answer the component parts, and then put them together to answer the broader question.

Here is a question:
{query}"""

embeddings = ZhipuAIEmbeddings()
prompt_templates = [physics_template, math_template]
prompt_embeddings = embeddings.embed_documents(prompt_templates)


def prompt_router(input):
    query_embedding = embeddings.embed_query(input["query"])
    similarity = cosine_similarity([query_embedding], prompt_embeddings)[0]
    most_similar = prompt_templates[similarity.argmax()]
    print("Using MATH" if most_similar == math_template else "Using PHYSICS")
    return PromptTemplate.from_template(most_similar)


chain = (
    {"query": RunnablePassthrough()}
    | RunnableLambda(prompt_router)
    | model
    | StrOutputParser()
)

In [3]:
print(chain.invoke("What's a black hole"))

Using PHYSICS
A black hole is a region of spacetime from which nothing, including light, can escape. It forms when a massive object collapses to such a small volume that its gravitational pull becomes infinitely strong. This extreme gravity means that anything approaching the black hole, including light, is pulled toward its center with such force that it can't escape.\n\nBlack holes come in various sizes and are classified based on their mass and charge. The largest and most massive black holes are found at the center of galaxies, while smaller ones can be found throughout the universe. Black holes are among the most extreme objects in physics, with their properties challenging our understanding of gravity and spacetime.


In [4]:
print(chain.invoke("What's a path integral"))

Using MATH
As an AI language model, I'm happy to help you understand path integrals. A path integral is a mathematical concept originating in physics, particularly in the field of quantum mechanics. It is used to describe the probability of a system evolving from one state to another through a series of intermediate states. The concept is often illustrated using the metaphor of a path, hence the name \"path integral.\"\n\nTo explain path integrals more formally, let's start with the quantum mechanical formalism. In quantum mechanics, the probability of a system evolving from a initial state |Ψ⟩ to a final state |Φ⟩ can be represented as:\n\nP(Ψ→Φ) = |⟨Φ|Ψ⟩|²\n\nwhere |Ψ⟩ and |Φ⟩ are column vectors containing the wave functions of the initial and final states, respectively, and |⟨Φ|Ψ⟩| is the scalar product (also known as the inner product) between these vectors.\n\nNow, imagine that the system evolves through a series of intermediate states |Ψ_i⟩ along the way, forming a path. The prob